In [31]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import re
import sqlparse
from os.path import abspath
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import os
import warnings
import findspark
import tensorflow as tf
import keras
from keras import layers

from keras.models import Model
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import expr
from pyspark.sql.functions import when, col

In [ ]:
findspark.init("C:\recomendation_systems\spark")
warnings.filterwarnings('ignore')

warehouse_location = abspath('spark-warehouse')


os.environ["SPARK_HOME"] = r"C:\recomendation_systems\spark"
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-20"
os.environ["HADOOP_HOMe"]=r"C:\recomendation_systems\spark\hadoop\bin"

In [ ]:
conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set('spark.driver.extraClassPath', r"C:\recomendation_systems\sqljdbc_12.4")
conf.set('spark.executor.extraClassPath', r"C:\recomendation_systems\sqljdbc_12.4")
spark = SparkSession.builder\
        .config(conf=conf)\
        .config("spark.sql.warehouse.dir", warehouse_location)\
        .getOrCreate()

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
df = pd.read_csv(r"C:\recomendation_systems\querys_treino.csv", sep=";", encoding="latin-1") 

In [22]:
df.columns

Index(['codigo', 'tipo', 'funcao'], dtype='object')

,codigo
0,"df.select(""firstname"",""lastname"").show()"
1,"df.select(df.firstname,df.lastname).show()"
2,"df.select(df[""firstname""],df[""lastname""]).show()"
3,from pyspark.sql.functions import col
4,"df.select(col(""firstname""),col(""lastname"")).sh..."


In [ ]:
import sqlparse


def extract_sql_columns(sql_query):
    parsed = sqlparse.parse(sql_query)
    if not parsed:
        return []

    for stmt in parsed:
        if stmt.get_type() == 'SELECT':
            select_stmt = stmt
            break
    else:
        return []

 
    columns = []
    for token in select_stmt.tokens:
        if isinstance(token, sqlparse.sql.IdentifierList):
            columns.extend([str(ident) for ident in token.get_identifiers()])
        elif isinstance(token, sqlparse.sql.Identifier):
            columns.append(str(token))
    
    return columns


In [32]:
"""Avalia querys e identifica """


nltk.download('stopwords')
nltk.download('wordnet')


base_treino = pd.read_csv("querys_treino.csv", sep=";", encoding="latin-1", usecols=['codigo', 'tipo', 'funcao'])
base_avaliacao = pd.read_csv("avaliaselectspark.csv", sep=";", encoding="latin-1", usecols=['codigo'])


spark_keywords = ['spark.sql', 'delta', 'df.select', 'join', 'df', 'pd']
sql_keywords = ['select', 'update', 'delete', 'merge', '%sql']


X_train = base_treino['codigo']
y_train = base_treino['tipo']

X_test = base_avaliacao['codigo']


def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    processed_text = ' '.join(words)

    processed_text = re.sub(r'([.,!?()])', r' \1 ', processed_text)

    return processed_text


X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)


vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()


def add_keywords_features(text, keywords):
    features = [1 if keyword in text else 0 for keyword in keywords]
    return features

X_train_keywords = X_train.apply(lambda x: add_keywords_features(x, spark_keywords + sql_keywords))
X_test_keywords = X_test.apply(lambda x: add_keywords_features(x, spark_keywords + sql_keywords))

X_train_tfidf = np.hstack((X_train_tfidf, np.array(X_train_keywords.tolist())))
X_test_tfidf = np.hstack((X_test_tfidf, np.array(X_test_keywords.tolist())))


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(256, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = build_model((X_train_tfidf.shape[1],), len(label_encoder.classes_))


model.fit(X_train_tfidf, y_train_encoded, epochs=2000, batch_size=128, verbose=2)


predictions = model.predict(X_test_tfidf)


y_test_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))


base_avaliacao['tipo_previsto'] = y_test_labels


print("Resultados:")
print(base_avaliacao[['codigo', 'tipo_previsto']])



Epoch 1/2000


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1/1 - 1s - loss: 0.6975 - accuracy: 0.5814 - 721ms/epoch - 721ms/step
Epoch 2/2000
1/1 - 0s - loss: 0.6826 - accuracy: 0.5814 - 11ms/epoch - 11ms/step
Epoch 3/2000
1/1 - 0s - loss: 0.6806 - accuracy: 0.5814 - 3ms/epoch - 3ms/step
Epoch 4/2000
1/1 - 0s - loss: 0.6832 - accuracy: 0.5581 - 0s/epoch - 0s/step
Epoch 5/2000
1/1 - 0s - loss: 0.6899 - accuracy: 0.5814 - 14ms/epoch - 14ms/step
Epoch 6/2000
1/1 - 0s - loss: 0.6795 - accuracy: 0.6047 - 4ms/epoch - 4ms/step
Epoch 7/2000
1/1 - 0s - loss: 0.6616 - accuracy: 0.6047 - 7ms/epoch - 7ms/step
Epoch 8/2000
1/1 - 0s - loss: 0.6715 - accuracy: 0.5814 - 4ms/epoch - 4ms/step
Epoch 9/2000
1/1 - 0s - loss: 0.6556 - accuracy: 0.5814 - 0s/epoch - 0s/step
Epoch 10/2000
1/1 - 0s - loss: 0.6549 - accuracy: 0.5814 - 9ms/epoch - 9ms/step
Epoch 11/2000
1/1 - 0s - loss: 0.6299 - accuracy: 0.6279 - 3ms/epoch - 3ms/step
Epoch 12/2000
1/1 - 0s - loss: 0.6264 - accuracy: 0.6279 - 0s/epoch - 0s/step
Epoch 13/2000
1/1 - 0s - loss: 0.6185 - accuracy: 0.6977 - 0

In [ ]:

nltk.download('stopwords')
nltk.download('wordnet')

base_treino = pd.read_csv("querys_treino.csv", sep=";", encoding="latin-1", usecols=['codigo', 'tipo', 'funcao'])
base_avaliacao = pd.read_csv("avaliaselectspark.csv", sep=";", encoding="latin-1", usecols=['codigo'])


spark_keywords = ['spark.sql', 'delta', 'df.select', 'join', 'df', 'pd']
sql_keywords = ['select', 'update', 'delete', 'merge', '%sql']


def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    processed_text = ' '.join(words)

    processed_text = re.sub(r'([.,!?()])', r' \1 ', processed_text)

    return processed_text

base_treino['codigo'] = base_treino['codigo'].apply(preprocess_text)
base_avaliacao['codigo'] = base_avaliacao['codigo'].apply(preprocess_text)


vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(base_treino['codigo']).toarray()
X_test_tfidf = vectorizer.transform(base_avaliacao['codigo']).toarray()


def add_keywords_features(text, keywords):
    features = [1 if keyword in text else 0 for keyword in keywords]
    return features

X_train_keywords = base_treino['codigo'].apply(lambda x: add_keywords_features(x, spark_keywords + sql_keywords))
X_test_keywords = base_avaliacao['codigo'].apply(lambda x: add_keywords_features(x, spark_keywords + sql_keywords))

X_train_tfidf = np.hstack((X_train_tfidf, np.array(X_train_keywords.tolist())))
X_test_tfidf = np.hstack((X_test_tfidf, np.array(X_test_keywords.tolist())))


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(base_treino['tipo'])


X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y_train_encoded, test_size=0.2, random_state=42)


def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(256, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model((X_train_tfidf.shape[1],), len(label_encoder.classes_))


model.fit(X_train, y_train, epochs=2000, batch_size=128, verbose=2)


predictions = model.predict(X_test)

y_test_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))


base_avaliacao['tipo_previsto'] = y_test_labels


print("Resultados:")
print(base_avaliacao[['codigo', 'tipo_previsto']])


def extract_sql_columns(sql_query):
    parsed = sqlparse.parse(sql_query)
    if not parsed:
        return []

    for stmt in parsed:
        if stmt.get_type() == 'SELECT':
            select_stmt = stmt
            break
    else:
        return []

    columns = []
    for token in select_stmt.tokens:
        if isinstance(token, sqlparse.sql.IdentifierList):
            columns.extend([str(ident) for ident in token.get_identifiers()])
        elif isinstance(token, sqlparse.sql.Identifier):
            columns.append(str(token))
    
    return columns


def evaluate_queries_and_identify(sql_query):
  
    pyspark_query = f"{}"
    pyspark_results = f"{}"

    if sql_query:
       
        sql_columns = extract_sql_columns(sql_query)
        
        pyspark_query = 'df.select(' + ', '.join([f'col("{col}")' for col in sql_columns]) + ')'
      
        pyspark_results = execute_sql_query(pyspark_query)

    return pyspark_query, pyspark_results




def execute_sql_query(sql_query):
    
 
    sql_query_from_csv = base_avaliacao[f"{}"].iloc[0]

    print("Executando consulta SQL:", sql_query_from_csv)


sql_query = f"{}"
pyspark_query, pyspark_results = evaluate_queries_and_identify(sql_query)


print("Consulta PySpark gerada:", pyspark_query)
print("Resultados da consulta PySpark:", pyspark_results)
